In [208]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 1. Загрузить данные из файла “ECDCCases.csv”.

In [209]:
data = pd.read_csv('ECDCCases.csv', sep=',')
print('size:', data.shape[0])
data.head()

size: 61904


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,14/12/2020,14,12,2020,746,6,Afghanistan,AF,AFG,38041757.0,Asia,9.013779
1,13/12/2020,13,12,2020,298,9,Afghanistan,AF,AFG,38041757.0,Asia,7.052776
2,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
3,12/12/2020,12,12,2020,113,11,Afghanistan,AF,AFG,38041757.0,Asia,6.868768
4,11/12/2020,11,12,2020,63,10,Afghanistan,AF,AFG,38041757.0,Asia,7.134266


# 2. Проверить в данных наличие пропущенных значений.
Вывести количество пропущенных значений в процентах. Удалить два признака, в которых больше всех пропущенных значений.
Для оставшихся признаков обработать пропуски: для категориального признака использовать заполнение
значением по умолчанию (например, «other»), для числового признака использовать заполнение медианным значением.
Показать, что пропусков больше в данных нет.

In [210]:
data.isna().sum()

dateRep                                                          0
day                                                              0
month                                                            0
year                                                             0
cases                                                            0
deaths                                                           0
countriesAndTerritories                                          0
geoId                                                          275
countryterritoryCode                                           123
popData2019                                                    123
continentExp                                                     0
Cumulative_number_for_14_days_of_COVID-19_cases_per_100000    2879
dtype: int64

In [227]:
def missing_pers(data_: pd.DataFrame):
    res_ = []
    for column in data_.columns:
        missing = np.mean(data_[column].isna()*100)
        res_.append((column, round(missing, 1)))
    return sorted(res_, key=lambda item: item[1], reverse=True)

In [228]:
def show_missed_pers(data_: pd.DataFrame):
    res_ = missing_pers(data_)
    for item in res_:
        print(f'{item[0]} : {item[1]}%')

In [229]:
show_missed_pers(data)

Cumulative_number_for_14_days_of_COVID-19_cases_per_100000 : 4.7%
geoId : 0.4%
countryterritoryCode : 0.2%
popData2019 : 0.2%
dateRep : 0.0%
day : 0.0%
month : 0.0%
year : 0.0%
cases : 0.0%
deaths : 0.0%
countriesAndTerritories : 0.0%
continentExp : 0.0%


In [214]:
cropped_data = data.drop([item[0] for item in missing_pers(data)[:2]], axis=1)
cropped_data.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,countryterritoryCode,popData2019,continentExp
0,14/12/2020,14,12,2020,746,6,Afghanistan,AFG,38041757.0,Asia
1,13/12/2020,13,12,2020,298,9,Afghanistan,AFG,38041757.0,Asia
2,12/12/2020,12,12,2020,113,11,Afghanistan,AFG,38041757.0,Asia
3,12/12/2020,12,12,2020,113,11,Afghanistan,AFG,38041757.0,Asia
4,11/12/2020,11,12,2020,63,10,Afghanistan,AFG,38041757.0,Asia


In [215]:
for col_name, type_ in cropped_data.dtypes.items():
    if type_.name == 'object':
        cropped_data[col_name].fillna('other', inplace=True)
    else:
        cropped_data[col_name].fillna(cropped_data[col_name].mean(), inplace=True)
cropped_data.isna().sum()

dateRep                    0
day                        0
month                      0
year                       0
cases                      0
deaths                     0
countriesAndTerritories    0
countryterritoryCode       0
popData2019                0
continentExp               0
dtype: int64

# 3. Посмотреть статистику по данным, используя describe().
Сделать выводы о том, какие признаки содержат выбросы. Посмотреть, для каких
стран количество смертей в день превысило 3000 и сколько таких дней было.

In [216]:
cropped_data.describe()

,day,month,year,cases,deaths,popData2019
count,61904.000000,61904.000000,61904.000000,61904.000000,61904.000000,6.190400e+04
mean,15.629232,7.067104,2019.998918,1155.079026,26.053987,4.098628e+07
std,8.841624,2.954816,0.032881,6779.010824,131.222948,1.529724e+08
min,1.000000,1.000000,2019.000000,-8261.000000,-1918.000000,8.150000e+02
25%,8.000000,5.000000,2020.000000,0.000000,0.000000,1.324820e+06
50%,15.000000,7.000000,2020.000000,15.000000,0.000000,7.813207e+06
75%,23.000000,10.000000,2020.000000,273.000000,4.000000,2.860872e+07
max,31.000000,12.000000,2020.000000,234633.000000,4928.000000,1.433784e+09


In [217]:
sample_: pd.DataFrame = cropped_data.loc[cropped_data.deaths > 3000]
print('days amount where deaths per day > 3000:', len(sample_))
sample_

days amount where deaths per day > 3000: 11


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,countryterritoryCode,popData2019,continentExp
2118,02/10/2020,2,10,2020,14001,3351,Argentina,ARG,44780675.0,America
16908,07/09/2020,7,9,2020,-8261,3800,Ecuador,ECU,17373657.0,America
37038,09/10/2020,9,10,2020,4936,3013,Mexico,MEX,127575529.0,America
44888,14/08/2020,14,8,2020,9441,3935,Peru,PER,32510462.0,America
44909,24/07/2020,24,7,2020,4546,3887,Peru,PER,32510462.0,America
59007,12/12/2020,12,12,2020,234633,3343,United_States_of_America,USA,329064917.0,America
59009,10/12/2020,10,12,2020,220025,3124,United_States_of_America,USA,329064917.0,America
59016,03/12/2020,3,12,2020,203311,3190,United_States_of_America,USA,329064917.0,America
59239,24/04/2020,24,4,2020,26543,3179,United_States_of_America,USA,329064917.0,America
59245,18/04/2020,18,4,2020,30833,3770,United_States_of_America,USA,329064917.0,America


# 4. Найти дублирование данных. Удалить дубликаты.

In [218]:
cropped_data[cropped_data.duplicated()]

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,countryterritoryCode,popData2019,continentExp
3,12/12/2020,12,12,2020,113,11,Afghanistan,AFG,38041757.0,Asia
218,12/05/2020,12,5,2020,285,2,Afghanistan,AFG,38041757.0,Asia
48010,29/05/2020,29,5,2020,0,0,Saint_Lucia,LCA,182795.0,America
48073,28/03/2020,28,3,2020,0,0,Saint_Lucia,LCA,182795.0,America


In [219]:
cropped_data.drop_duplicates(inplace=True)

# 5. Загрузить данные из файла “bmi.csv”.
Взять оттуда две выборки. Одна выборка – это индекс массы тела людей c региона northwest,
вторая выборка – это индекс массы тела людей с региона southwest. Сравнить средние значения
этих выборок, используя t-критерий Стьюдента. Предварительно проверить выборки на нормальность (критерий Шопиро-Уилка)
и на гомогенность дисперсии (критерий Бартлетта).

In [220]:
data_2 = pd.read_csv('bmi.csv', sep=',')
data_northwest: pd.DataFrame = data_2.loc[data_2.region == 'northwest']
data_southwest: pd.DataFrame = data_2.loc[data_2.region == 'southwest']

In [221]:
import scipy.stats as sts

In [222]:
n = 25
sample_nw = data_northwest.sample(n=n)
sample_sw = data_southwest.sample(n=n)
res_nw = sts.shapiro(sample_nw.bmi)
res_sw = sts.shapiro(sample_sw.bmi)
print(res_nw, '- имеет нормальное распределение' if res_nw.pvalue > 0.05 else 'не имеет нормального распределения')
print(res_sw, '- имеет нормальное распределение' if res_nw.pvalue > 0.05 else 'не имеет нормального распределения')
res = sts.bartlett(sample_nw.bmi, sample_sw.bmi)
print(res, '- дисперсии выборок примерно одинаковы' if res.pvalue > 0.05 else 'дисперсии выборок различны')
t_res = sts.ttest_ind(sample_nw.bmi, sample_sw.bmi)
print(t_res, '- выборки принадлежат одной генеральной совокупности' if t_res.pvalue > 0.05 else 'выборки не принадлежат одной генеральной совокупности')

ShapiroResult(statistic=0.9726725220680237, pvalue=0.713208019733429) - имеет нормальное распределение
ShapiroResult(statistic=0.9702517986297607, pvalue=0.6514624357223511) - имеет нормальное распределение
BartlettResult(statistic=0.43438345820190155, pvalue=0.5098458535854744) - дисперсии выборок примерно одинаковы
Ttest_indResult(statistic=0.34495630080069956, pvalue=0.7316342479053869) - выборки принадлежат одной генеральной совокупности


# 6. Кубик бросили 600 раз, получили следующие результаты:
| N   | Количество <br/>выпадений |
|:----|--------------------------:|
| 1   |                        97 |
| 2   |                        98 |
| 3   |                       109 |
| 4   |                        95 |
| 5   |                        97 |
| 6   |                       104 |

С помощью критерия Хи-квадрат проверить, является ли полученное распределение равномерным. Использовать функцию scipy.stats.chisquare().

In [223]:
D = pd.DataFrame({'points': [1, 2, 3, 4, 5, 6],'observed':[97, 98, 109, 95, 97, 104], })
D['expected'] = 100
D

,points,observed,expected
0,1,97,100
1,2,98,100
2,3,109,100
3,4,95,100
4,5,97,100
5,6,104,100


In [224]:
x_res = sts.chisquare(D.observed, D.expected)
print(x_res, '- распределение является равномерным' if t_res.pvalue > 0.05 else 'распределение не является равномерным')

Power_divergenceResult(statistic=1.44, pvalue=0.9198882077437889) - распределение является равномерным


# 7. С помощью критерия Хи-квадрат проверить, являются ли переменные зависимыми.
Создать дата фрейм, используя следующий код:
data = pd.DataFrame({
    'Женат': [89,17,11,43,22,1],
    'Гражданский брак': [80,22,20,35,6,4],
    'Не состоит в отношениях': [35,44,35,6,8,22]
})
data.index = ['Полный рабочий день','Частичная занятость','Временно не работает','На домохозяйстве','На пенсии','Учёба']
Использовать функцию scipy.stats.chi2_contingency().
Влияет ли семейное положение на занятость?

In [225]:
D_2 = pd.DataFrame({
    'Женат': [89,17,11,43,22,1],
    'Гражданский брак': [80,22,20,35,6,4],
    'Не состоит в отношениях': [35,44,35,6,8,22]
})
D_2.index = ['Полный рабочий день','Частичная занятость','Временно не работает','На домохозяйстве','На пенсии','Учёба']
D_2

,Женат,Гражданский брак,Не состоит в отношениях
Полный рабочий день,89,80,35
Частичная занятость,17,22,44
Временно не работает,11,20,35
На домохозяйстве,43,35,6
На пенсии,22,6,8
Учёба,1,4,22


In [226]:
x2_res = sts.chi2_contingency(D_2)[:3]
print(x2_res, '- переменные являются зависимыми' if x2_res[1] < 0.05 else 'переменные не являются зависимыми')

(122.29654948595365, 1.7291616900960234e-21, 10) - переменные являются зависимыми
